In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install keras-nlp --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.1/590.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 67.1 MB/s eta 0:00:00


In [3]:
cd "/content/drive/MyDrive/Datasets/paragraphs"

/content/drive/MyDrive/Datasets/paragraphs


In [4]:
import numpy as np
import tensorflow as tf
import keras
from keras import Sequential
from keras.layers import *
from keras.layers.experimental import preprocessing
from keras.preprocessing.text import Tokenizer
import keras_nlp
from sklearn.model_selection import train_test_split

Using TensorFlow backend


In [5]:
TXTDataset = "paragraphs.txt"

In [6]:
with open(TXTDataset,"r",encoding="utf-8") as folder:
  folder_content = folder.read()

paragraphic = sorted(set(folder_content))
len(paragraphic)

93

In [7]:
with open(TXTDataset, "r", encoding="utf-8") as file:
    file_contents = file.read()

paragraphs = file_contents.split("\n")

In [8]:
myDataDataset = np.array(paragraphs)

In [9]:
def DataLabeling(prelabeledData):
  YLabels = []

  for i in prelabeledData:
    YLabels.append(float("{:.2f}".format(len(i)/200)))

  return YLabels

In [10]:
YDataset = np.array(DataLabeling(myDataDataset))

In [ ]:
testa = iter(YDataset)

In [11]:
tokenizer = keras_nlp.models.BertTokenizer.from_preset(
    "bert_base_en_uncased",
)

231508/231508 [==============================] - 0s 2us/step


In [12]:
Dataset = tokenizer(myDataDataset)

In [23]:
tf.compat.v1.disable_eager_execution()

In [ ]:
Dataset.num_words

In [10]:
chars = tf.strings.unicode_split(myDataDataset, input_encoding='UTF-8')

In [ ]:
asr = iter(chars)

In [ ]:
next(asr)

In [11]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(paragraphic), mask_token=None)

In [ ]:
ids = ids_from_chars(chars)
ids[0]

In [13]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
chars = chars_from_ids(ids[0])
chars

In [15]:
tf.strings.reduce_join(chars, axis=-1).numpy()

b'This biographical article related to French artistic gymnastics is a stub. You can help Wikipedia by expanding it.'

In [16]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [102]:
Dataset = ids_from_chars(tf.strings.unicode_split(np.array(paragraphs), 'UTF-8'))

NameError: ignored

In [19]:
print(YDataset.shape,myDataDataset.shape)

(31613,) (31613,)


In [20]:
print((np.array([0,0,0,0,0,0,0,0]) == 0).all())

True


In [13]:
dataset = tf.data.Dataset.from_tensor_slices((Dataset.to_tensor(), YDataset))

In [ ]:
for i in dataset.take(2):
  print(i)

In [23]:
batch_size = 64

In [14]:
dataset = (
    dataset
    .shuffle(10000)
    .batch(128, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(128, 165), dtype=tf.int32, name=None), TensorSpec(shape=(128,), dtype=tf.float64, name=None))>

### **This was the last code with GRU and OOP**

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.Embedding = keras.layers.Embedding(len(paragraphic), embedding_dim)
    self.gru = keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.gru2 = keras.layers.GRU(64,return_sequences=False,
                                   return_state=False)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.tokenizer = Tokenizer()

  def call(self, inputs, states=None, return_state=False, training=False):
    for XData,YData in enumerate(inputs):
      input = XData
      label = YData
      # print("--------------------------------------------------\n",XData,YData , "These are the data being given \n")
      x = input
      x = self.Embedding(x)
      x = keras.layers.Reshape((1, -1))(x)
      if states is None:
        states = self.gru.get_initial_state(x)
      x, states = self.gru(x, initial_state=states, training=training)
      x, states = self.gru2(x)
      x = keras.layers.Reshape((32,1))(x)
      x = Dense(1,activation="relu")(x)
      x = Flatten()(x)

    # x = inputs


    # if return_state:
    #   return x, states
    # else:
    #   return x

    return x,states if return_state else x

In [ ]:
# @title Default title text
for epoch in range(epochs):

    print(f'Epoch {epoch+1}/{epochs}')
    for i,batch in enumerate(dataset.batch(batch_size)):
        X_batch, y_batch = batch

        with tf.GradientTape() as tape:
              # print(X_batch.shape , "THis is the shape of batch")
              predictions = model(X_batch , training=True)
              # print("----------------------------------------------------\n\n",predictions[0] , " : This is the prediction being made \n \n --------------------------------")
              # print(np.array(predictions).shape , "This is the predictions shape")
              # print(y_batch)
              print(np.array(y_batch).shape,"\n",np.array(predictions).shape)
              loss = model.loss(y_batch, predictions)


        gradients = tape.gradient(loss, model.trainable_variables)

        model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        if i % 100 == 0:
            print(f'Step {i}/{len(dataset)} - Loss: {loss:.4f}')

In [ ]:
len(paragraphic)

## **This is the normal model being created**

In [1]:
model = keras.Sequential()


# model.add(keras.layers.Reshape((1, -1)))

# model.add(tf.keras.layers.SimpleRNN(units=1024, return_sequences=False, return_state=False))
# gru = tf.keras.layers.GRU(1024, return_sequences=True, return_state=True)
# whole_sequence_output, final_state = gru(inputs)
# model.add(Input(shape=(1)))
# model.add(Embedding(31139,512))
model.add(Embedding(632508,512,input_shape=(None,)))
model.add(Bidirectional(units=256,return_sequences=False,return_state=False))
model.add(Flatten())

model.add(keras.layers.Dense(1, activation="softmax"))

model.summary()

NameError: ignored

In [18]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.003),loss=keras.losses.MeanSquaredLogarithmicError(),metrics=["accuracy"])

In [21]:
history = model.fit(dataset,epochs=2)

RuntimeError: ignored

In [30]:
LoremIpsum_text = """Lorem ipsum dolo
r sit amet, consectetur adipiscing el
it. Pellentesque quis hendrerit justo.
 In sed aliquam tellus. Ut auctor nisl sit amet dui cursus, quis hendrerit diam ultrices. Donec aliquet, sapien a cursus ullamcorper, eros velit conval
 lis tortor, eu vestibulum turpis dolor ac justo. Vivamus a volutpat mauris, in ultricies arcu. Vestibulum bibendum arcu non sapien fringilla, ac tempus purus vulputate. Suspendisse semper augue vitae odio consequat, ut
 volutpat dolor suscipit. Vestibulum elementum quam et tellus feugiat, in cursus tortor vehicula. Integer luctus, dolor eget aliquam aliquam, odio justo cursus urna, in cursus tellus est a felis. Vivamus sed euismod arcu. Prae
 sent sit amet lobortis leo, a facilisis dui. Ut rhoncus ut libero id tempor. Integer condimentum, odio eget ullamcorper cursus, urna nisi placerat nisl, vel dictum metus dui quis tortor."""*5
print(len(LoremIpsum_text))
# Generate a 512-character paragraph
generated_paragraph = LoremIpsum_text[:512]

4405


In [28]:
predictions = model.predict(generated_paragraph)

IndexError: ignored

In [55]:
print(np.squeeze(np.array([[1,2],[2,512]])))

[[  1   2]
 [  2 512]]


In [49]:
train_ratio = 0.8
train_size = int(len(list(dataset)) * train_ratio)

train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

RuntimeError: ignored

In [94]:
print(dataset.take(1))

<_TakeDataset element_spec=(RaggedTensorSpec(TensorShape([512, None]), tf.int32, 1, tf.int64), TensorSpec(shape=(512,), dtype=tf.float64, name=None))>
